#**Pest detection using TensorFlow Lite Model Maker**

### **Install and import libraries**

In [ ]:
# Create main directory
!mkdir TF

In [ ]:
# Install libraries
!pip install -q tflite-model-maker
!pip install -q pycocotools
!pip install -q cython

In [ ]:
# Get requirements
#%cd /content
#!wget https://raw.githubusercontent.com/tensorflow/examples/master/tensorflow_examples/lite/model_maker/requirements.txt

In [ ]:
#!cat /content/requirements.txt

In [ ]:
#!pip install --ignore-installed -r /content/requirements.txt  

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import pathlib
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import glob
import argparse
import xml.etree.ElementTree as ET
from tqdm import tqdm
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import random, shutil
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tflite_model_maker as mm
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
# Checking versions
print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

TensorFlow Version: 2.6.0
Model Maker Version: 0.3.2


In [ ]:
# Setting a seed for reproducibility
tf.random.set_seed(123)

### **Getting the data**

Training, validation, and test datasets can be found as a zip  [here](https://drive.google.com/drive/folders/1q38SziaAaH4AW8dFI4wzIwL3yqnPj_4i?usp=sharing), you can create a shortcut between your Drive account and this Drive folder to better access it. 

After getting a test dataset in the data extraction notebook, I used the free and quick service of the platform [Roboflow](https://blog.roboflow.com/create-tfrecord/) to partition this data and convert it into TFRecords files. The reason to use this dataset is that apparently, TFLite Model Maker has a limit in the number of training images when the data is uploaded as a TFRecord file.

So I will not use the original datasets but the files created by Roboflow. The only thing I will change from these files is the label map to make it more readable.

In [ ]:
%%bash

# Create new directories for better organization
cd /content/TF
mkdir -p workspace/training/{images,model,exported-models,annotations}  

# Download data
cd /content/TF/workspace/training/images
unzip /content/drive/MyDrive/OMDENA/small_dataset.v1i.tfrecord.zip

Archive:  /content/drive/MyDrive/OMDENA/small_dataset.v1i.tfrecord.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/pests.tfrecord     
 extracting: test/pests_label_map.pbtxt  
   creating: train/
 extracting: train/pests.tfrecord    
 extracting: train/pests_label_map.pbtxt  
   creating: valid/
 extracting: valid/pests.tfrecord    
 extracting: valid/pests_label_map.pbtxt  


##### Now we need to modify the `pests_label_map.pbtxt` file, to specify the classes, and save them into annnotations folder. 


In [ ]:
%cd /content/TF/workspace/training/annotations
!wget https://raw.githubusercontent.com/Camicb/tmp/main/pests_label_map.pbtxt

/content/TF/workspace/training/annotations
--2021-09-21 15:15:55--  https://raw.githubusercontent.com/Camicb/tmp/main/pests_label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1113 (1.1K) [text/plain]
Saving to: ‘pests_label_map.pbtxt’

pests_label_map.pbt 100%[===================>]   1.09K  --.-KB/s    in 0s      

2021-09-21 15:15:55 (56.5 MB/s) - ‘pests_label_map.pbtxt’ saved [1113/1113]



### **Modeling with TFLite Model Maker**

Model Maker supports the EfficientDet-Lite family of object detection models. (EfficientDet-Lite is derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html), which offers state-of-the-art accuracy in a small model size). There are several model sizes we can choose from:

|| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|-|--------------------|-----------|---------------|----------------------|
|| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
|| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
|| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
|| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
|| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |
| <td colspan=4><br><i>* File size of the integer quantized models. <br/>** Latency measured on Pixel 4 using 4 threads on CPU. <br/>*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.</i></td> |


We do not need to resize the images or perform data augmentation steps, because the models already have the built-in mechanisms to do so. Also, since we have small datasets, I will set `train_whole_model=True`. I will try Lite1, Lite2 and Lite3 models.

Next, I will export the models to the TensorFlow Lite format. By default, the [`export()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#export) method performs [full integer post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization). Model Maker uses the same dataset we gave to our model spec as a representative dataset, which is required for full-int quantization; we just need to specify the export directory and format. By default, it exports to TF Lite, but we also want a labels file, so I will declare both.

In [ ]:
# Load the datasets into Model Maker

model_dir = '/content/TF/workspace/training/model' 

tf_record_train = '/content/TF/workspace/training/images/train/pests.tfrecord'
tf_record_val =   '/content/TF/workspace/training/images/valid/pests.tfrecord'
tf_record_test = '/content/TF/workspace/training/images/test/pests.tfrecord'

label_map = ['1','4','5','10','16','23','24','28','29','30','33','34','14111','14131','14141']

train_data = object_detector.DataLoader(tf_record_train, 97, label_map)  # can't be more than 100 images
val_data = object_detector.DataLoader(tf_record_val, 15, label_map)
test_data = object_detector.DataLoader(tf_record_test, 15, label_map)

**EfficientDet-Lite1**

In [ ]:
# Choose the object detection model architecture
spec = object_detector.EfficientDetLite1Spec()

# Train the model
model_lite1 = object_detector.create(train_data=train_data, model_spec=spec, validation_data=val_data, batch_size=8, train_whole_model=True)

Epoch 1/50
12/12 [==============================] - 91s 2s/step - det_loss: 2.1872 - cls_loss: 1.3112 - box_loss: 0.0175 - reg_l2_loss: 0.0730 - loss: 2.2602 - learning_rate: 0.0090 - gradient_norm: 1.4924 - val_det_loss: 1.9579 - val_cls_loss: 1.2878 - val_box_loss: 0.0134 - val_reg_l2_loss: 0.0730 - val_loss: 2.0309
Epoch 2/50
12/12 [==============================] - 11s 955ms/step - det_loss: 2.1171 - cls_loss: 1.3158 - box_loss: 0.0160 - reg_l2_loss: 0.0730 - loss: 2.1900 - learning_rate: 0.0100 - gradient_norm: 1.4055 - val_det_loss: 1.9310 - val_cls_loss: 1.2868 - val_box_loss: 0.0129 - val_reg_l2_loss: 0.0730 - val_loss: 2.0040
Epoch 3/50
12/12 [==============================] - 11s 969ms/step - det_loss: 2.0552 - cls_loss: 1.2958 - box_loss: 0.0152 - reg_l2_loss: 0.0730 - loss: 2.1281 - learning_rate: 0.0099 - gradient_norm: 1.6441 - val_det_loss: 1.9115 - val_cls_loss: 1.2877 - val_box_loss: 0.0125 - val_reg_l2_loss: 0.0730 - val_loss: 1.9845
Epoch 4/50
12/12 [================

In [ ]:
# Evaluate the model 
model_lite1.evaluate(test_data)

1/1 [==============================] - 11s 11s/step



{'AP': 0.14520594,
 'AP50': 0.25615564,
 'AP75': 0.15242714,
 'AP_/1': 0.055263158,
 'AP_/10': 0.118937954,
 'AP_/14111': 0.0,
 'AP_/14131': 0.0,
 'AP_/14141': 0.0,
 'AP_/16': 0.0060741375,
 'AP_/23': 0.26666668,
 'AP_/24': 0.12886967,
 'AP_/28': 0.046883468,
 'AP_/29': 0.08415841,
 'AP_/30': 0.0,
 'AP_/33': 0.0,
 'AP_/34': 0.0,
 'AP_/4': 0.0,
 'AP_/5': 0.6,
 'APl': 0.17684014,
 'APm': 0.08992086,
 'APs': -1.0,
 'ARl': 0.45185184,
 'ARm': 0.18333334,
 'ARmax1': 0.26740742,
 'ARmax10': 0.4048148,
 'ARmax100': 0.40814814,
 'ARs': -1.0}

Precision defines how much you can rely on the positive class prediction: From the samples that the model said were positive, how many actually are? (FP)

The Recall measures how good the model is at hitting the positive class, That is, from the positive samples, how many did the algorithm get right? (FN)

The Average Recall(AR) was split by the max number of detection per image (1, 10, 100).

In [ ]:
# Export the model
model_lite1.export(export_dir='/content',
             tflite_filename='model_lite1.tflite',
             label_filename='model_lite1-labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

**EfficientDet-Lite2**

In [ ]:
# Choose the object detection model architecture
spec = object_detector.EfficientDetLite2Spec()

# Train the model
model_lite2 = object_detector.create(train_data=train_data, model_spec=spec, validation_data=val_data, batch_size=8, train_whole_model=True)

Epoch 1/50
12/12 [==============================] - 68s 2s/step - det_loss: 2.1144 - cls_loss: 1.3493 - box_loss: 0.0153 - reg_l2_loss: 0.0782 - loss: 2.1926 - learning_rate: 0.0090 - gradient_norm: 1.5039 - val_det_loss: 2.0255 - val_cls_loss: 1.3333 - val_box_loss: 0.0138 - val_reg_l2_loss: 0.0782 - val_loss: 2.1037
Epoch 2/50
12/12 [==============================] - 15s 1s/step - det_loss: 2.0865 - cls_loss: 1.3625 - box_loss: 0.0145 - reg_l2_loss: 0.0782 - loss: 2.1647 - learning_rate: 0.0100 - gradient_norm: 1.6573 - val_det_loss: 1.9846 - val_cls_loss: 1.3252 - val_box_loss: 0.0132 - val_reg_l2_loss: 0.0782 - val_loss: 2.0628
Epoch 3/50
12/12 [==============================] - 15s 1s/step - det_loss: 2.0057 - cls_loss: 1.3394 - box_loss: 0.0133 - reg_l2_loss: 0.0782 - loss: 2.0839 - learning_rate: 0.0099 - gradient_norm: 2.0190 - val_det_loss: 1.9615 - val_cls_loss: 1.3212 - val_box_loss: 0.0128 - val_reg_l2_loss: 0.0782 - val_loss: 2.0397
Epoch 4/50
12/12 [======================

In [ ]:
# Evaluate the model 
model_lite2.evaluate(test_data)

1/1 [==============================] - 11s 11s/step



{'AP': 0.20116763,
 'AP50': 0.35209695,
 'AP75': 0.19949001,
 'AP_/1': 0.73333335,
 'AP_/10': 0.07293909,
 'AP_/14111': 0.0,
 'AP_/14131': 0.0,
 'AP_/14141': 0.0,
 'AP_/16': 0.26930693,
 'AP_/23': 0.225,
 'AP_/24': 0.14992929,
 'AP_/28': 0.36,
 'AP_/29': 0.0,
 'AP_/30': 0.0,
 'AP_/33': 0.0,
 'AP_/34': 0.0,
 'AP_/4': 0.0,
 'AP_/5': 0.0,
 'APl': 0.2750511,
 'APm': 0.047805496,
 'APs': -1.0,
 'ARl': 0.40555555,
 'ARm': 0.1,
 'ARmax1': 0.26962963,
 'ARmax10': 0.34296295,
 'ARmax100': 0.34407407,
 'ARs': -1.0}

In [ ]:
# Export the model
model_lite2.export(export_dir='/content',
             tflite_filename='model_lite2.tflite',
             label_filename='model_lite2-labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

**EfficientDet-Lite3**

In [ ]:
# Choose the object detection model architecture
spec = object_detector.EfficientDetLite3Spec()

# Train the model
model_lite3 = object_detector.create(train_data=train_data, model_spec=spec, validation_data=val_data, batch_size=8, train_whole_model=True)

Epoch 1/50
12/12 [==============================] - 86s 2s/step - det_loss: 2.0739 - cls_loss: 1.3179 - box_loss: 0.0151 - reg_l2_loss: 0.0949 - loss: 2.1687 - learning_rate: 0.0090 - gradient_norm: 1.1736 - val_det_loss: 1.9253 - val_cls_loss: 1.2985 - val_box_loss: 0.0125 - val_reg_l2_loss: 0.0949 - val_loss: 2.0202
Epoch 2/50
12/12 [==============================] - 22s 2s/step - det_loss: 2.0338 - cls_loss: 1.3213 - box_loss: 0.0143 - reg_l2_loss: 0.0949 - loss: 2.1287 - learning_rate: 0.0100 - gradient_norm: 1.3204 - val_det_loss: 1.8967 - val_cls_loss: 1.3001 - val_box_loss: 0.0119 - val_reg_l2_loss: 0.0949 - val_loss: 1.9916
Epoch 3/50
12/12 [==============================] - 22s 2s/step - det_loss: 1.9854 - cls_loss: 1.3141 - box_loss: 0.0134 - reg_l2_loss: 0.0949 - loss: 2.0802 - learning_rate: 0.0099 - gradient_norm: 1.6296 - val_det_loss: 1.8604 - val_cls_loss: 1.3016 - val_box_loss: 0.0112 - val_reg_l2_loss: 0.0949 - val_loss: 1.9553
Epoch 4/50
12/12 [======================

In [ ]:
# Evaluate the model
model_lite3.evaluate(test_data)

1/1 [==============================] - 13s 13s/step



{'AP': 0.17297256,
 'AP50': 0.3468467,
 'AP75': 0.120462045,
 'AP_/1': 0.05,
 'AP_/10': 0.05797995,
 'AP_/14111': 0.0,
 'AP_/14131': 0.0,
 'AP_/14141': 0.0,
 'AP_/16': 0.05049505,
 'AP_/23': 0.9,
 'AP_/24': 0.011043247,
 'AP_/28': 0.3,
 'AP_/29': 0.033663366,
 'AP_/30': 0.0,
 'AP_/33': 0.0,
 'AP_/34': 0.0,
 'AP_/4': 0.0,
 'AP_/5': 0.15357143,
 'APl': 0.2066049,
 'APm': 0.02084844,
 'APs': -1.0,
 'ARl': 0.31851852,
 'ARm': 0.090277776,
 'ARmax1': 0.25296298,
 'ARmax10': 0.28574073,
 'ARmax100': 0.28685185,
 'ARs': -1.0}

In [ ]:
# Export the model
model_lite3.export(export_dir='/content',
             tflite_filename='model_lite3.tflite',
             label_filename='model_lite3-labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

### **Evaluate the TF Lite model**

Exporting the model to TensorFlow Lite can affect the model accuracy, due to the reduced numerical precision from quantization and because the original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TF Lite model uses global NMS, which is faster but less accurate.

Therefore you should always evaluate the exported TF Lite model and be sure it still meets your requirements:

In [ ]:
%cd /content
model_lite2.evaluate_tflite('/content/model_lite2.tflite', test_data)  # apparently only works with tf==2.5 and currently is 2.6

/content


InvalidArgumentError: ignored

Other resources:

- https://www.tensorflow.org/lite/tutorials/model_maker_object_detection
- https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_efficientdet_model_maker_tf2.ipynb#scrollTo=CgCDMe0e6jlT